# Step 3 — Feature Engineering

This notebook performs advanced feature engineering to create powerful predictive features from the cleaned dataset. Feature engineering is critical for improving model performance by creating meaningful representations of the data.

## Planned Features:

1. **Tenure-based features:**
   - `tenure_in_years`: Convert tenure from months to years for easier interpretation
   - `tenure_group`: Categorical grouping (New: 0-12 months, Medium: 13-48 months, Long: 49+ months)

2. **Ratio features:**
   - `monthly_charges_to_tenure_ratio`: Monthly charges divided by tenure (months)
   - `total_charges_per_month`: Alternative calculation of average monthly spend
   - `charges_vs_avg_ratio`: How customer's charges compare to the average

3. **Binary flags:**
   - `has_dependents`: Binary flag for customers with dependents
   - `is_senior_citizen`: Already exists but ensure it's properly formatted
   - `has_phone_service`: Binary flag for phone service
   - `has_internet_service`: Binary flag for any internet service
   - `has_premium_services`: Flag for customers with multiple premium add-ons

4. **Service complexity features:**
   - `total_services`: Count of additional services (OnlineSecurity, OnlineBackup, etc.)
   - `service_utilization_score`: Weighted score based on service usage
   - `payment_risk_score`: Risk score based on payment method and contract type

5. **Interaction features:**
   - `contract_charges_interaction`: Interaction between contract type and monthly charges
   - `internet_charges_interaction`: Interaction between internet service type and charges

The engineered dataset will be saved as `data/featured_telco_churn.csv` for modeling.

In [1]:
# Imports and setup
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 180)

# Load the cleaned dataset
data_path = Path('..') / 'data' / 'cleaned_telco_churn.csv'
df = pd.read_csv(data_path)

print(f"Loaded dataset: {data_path}")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()[:10]}...")  # Show first 10 columns

# Display basic info
df.head()

Loaded dataset: ../data/cleaned_telco_churn.csv
Shape: (7032, 47)
Columns: ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'gender_Female', 'gender_Male', 'Partner_No', 'Partner_Yes', 'Dependents_No', 'Dependents_Yes']...


,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_No,Churn_Yes
0,0,1,29.85,29.85,True,False,False,True,True,False,True,False,False,True,False,True,False,False,True,False,False,False,False,True,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,True,False,True,False
1,0,34,56.95,1889.50,False,True,True,False,True,False,False,True,True,False,False,True,False,False,False,False,True,True,False,False,False,False,True,True,False,False,True,False,False,True,False,False,False,True,False,True,False,False,False,False,True,True,False
2,0,2,53.85,108.15,False,True,True,False,True,False,False,True,True,False,False,True,False,False,False,False,True,False,False,True,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,False,True,False,True
3,0,45,42.30,1840.75,False,True,True,False,True,False,True,False,False,True,False,True,False,False,False,False,True,True,False,False,False,False,True,False,False,True,True,False,False,True,False,False,False,True,False,True,False,True,False,False,False,True,False
4,0,2,70.70,151.65,True,False,True,False,True,False,False,True,True,False,False,False,True,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,True,False,False,True


In [2]:
# 1. TENURE-BASED FEATURES

# Convert tenure to years
df['tenure_in_years'] = df['tenure'] / 12

# Create tenure groups
def categorize_tenure(tenure_months):
    if tenure_months <= 12:
        return 'New'
    elif tenure_months <= 48:
        return 'Medium'
    else:
        return 'Long'

df['tenure_group'] = df['tenure'].apply(categorize_tenure)

# 2. RATIO FEATURES

# Monthly charges to tenure ratio (higher values may indicate higher churn risk)
# Add small constant to avoid division by zero
df['monthly_charges_to_tenure_ratio'] = df['MonthlyCharges'] / (df['tenure'] + 1)

# Total charges per month (alternative calculation)
df['total_charges_per_month'] = df['TotalCharges'] / (df['tenure'] + 1)

# How customer charges compare to average
avg_monthly_charges = df['MonthlyCharges'].mean()
df['charges_vs_avg_ratio'] = df['MonthlyCharges'] / avg_monthly_charges

print("Created tenure and ratio features:")
print(f"tenure_in_years: {df['tenure_in_years'].describe()}")
print(f"tenure_group distribution: \n{df['tenure_group'].value_counts()}")
print(f"monthly_charges_to_tenure_ratio: {df['monthly_charges_to_tenure_ratio'].describe()}")

Created tenure and ratio features:
tenure_in_years: count    7032.000000
mean        2.701816
std         2.045438
min         0.083333
25%         0.750000
50%         2.416667
75%         4.583333
max         6.000000
Name: tenure_in_years, dtype: float64
tenure_group distribution: 
tenure_group
Medium    2618
Long      2239
New       2175
Name: count, dtype: int64
monthly_charges_to_tenure_ratio: count    7032.000000
mean        5.714882
std         8.567435
min         0.264384
25%         1.250000
50%         2.073598
75%         5.884842
max        51.225000
Name: monthly_charges_to_tenure_ratio, dtype: float64


In [3]:
# 3. BINARY FLAGS

# Extract binary features from encoded columns
def extract_binary_flag(df, prefix, positive_value):
    """Extract binary flag from one-hot encoded columns"""
    pos_col = f"{prefix}_{positive_value}"
    if pos_col in df.columns:
        return df[pos_col]
    else:
        # Handle case where column might be named differently
        matching_cols = [col for col in df.columns if col.startswith(prefix) and positive_value in col]
        if matching_cols:
            return df[matching_cols[0]]
        else:
            print(f"Warning: Could not find column for {prefix}_{positive_value}")
            return 0

# Create binary flags
df['has_dependents'] = extract_binary_flag(df, 'Dependents', 'Yes')
df['has_phone_service'] = extract_binary_flag(df, 'PhoneService', 'Yes')

# Internet service flag (any internet service)
internet_cols = [col for col in df.columns if col.startswith('InternetService_') and col != 'InternetService_No']
if internet_cols:
    df['has_internet_service'] = df[internet_cols].sum(axis=1).clip(0, 1)
else:
    df['has_internet_service'] = 0

# Already have SeniorCitizen, but ensure it's binary
df['is_senior_citizen'] = df['SeniorCitizen']

print("Created binary flags:")
print(f"has_dependents: {df['has_dependents'].value_counts()}")
print(f"has_phone_service: {df['has_phone_service'].value_counts()}")
print(f"has_internet_service: {df['has_internet_service'].value_counts()}")
print(f"is_senior_citizen: {df['is_senior_citizen'].value_counts()}")

Created binary flags:
has_dependents: has_dependents
False    4933
True     2099
Name: count, dtype: int64
has_phone_service: has_phone_service
True     6352
False     680
Name: count, dtype: int64
has_internet_service: has_internet_service
1    5512
0    1520
Name: count, dtype: int64
is_senior_citizen: is_senior_citizen
0    5890
1    1142
Name: count, dtype: int64


In [4]:
# 4. SERVICE COMPLEXITY FEATURES

# Count additional services (premium add-ons)
premium_services = []
service_prefixes = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

for prefix in service_prefixes:
    yes_col = f"{prefix}_Yes"
    if yes_col in df.columns:
        premium_services.append(yes_col)
    else:
        # Try to find the column with a different naming pattern
        matching_cols = [col for col in df.columns if col.startswith(prefix) and 'Yes' in col]
        if matching_cols:
            premium_services.append(matching_cols[0])

print(f"Found premium service columns: {premium_services}")

# Total services count
if premium_services:
    df['total_services'] = df[premium_services].sum(axis=1)
else:
    df['total_services'] = 0

# Has premium services flag
df['has_premium_services'] = (df['total_services'] > 0).astype(int)

# Service utilization score (weighted by importance/cost)
service_weights = {
    'OnlineSecurity': 1.5,
    'OnlineBackup': 1.0,
    'DeviceProtection': 1.2,
    'TechSupport': 1.8,
    'StreamingTV': 1.0,
    'StreamingMovies': 1.0
}

df['service_utilization_score'] = 0
for service_prefix in service_prefixes:
    yes_col = f"{service_prefix}_Yes"
    if yes_col in df.columns:
        weight = service_weights.get(service_prefix, 1.0)
        df['service_utilization_score'] += df[yes_col] * weight

print("Created service complexity features:")
print(f"total_services: {df['total_services'].describe()}")
print(f"has_premium_services: {df['has_premium_services'].value_counts()}")
print(f"service_utilization_score: {df['service_utilization_score'].describe()}")

Found premium service columns: ['OnlineSecurity_Yes', 'OnlineBackup_Yes', 'DeviceProtection_Yes', 'TechSupport_Yes', 'StreamingTV_Yes', 'StreamingMovies_Yes']
Created service complexity features:
total_services: count    7032.000000
mean        2.038111
std         1.847161
min         0.000000
25%         0.000000
50%         2.000000
75%         3.000000
max         6.000000
Name: total_services, dtype: float64
has_premium_services: has_premium_services
1    4819
0    2213
Name: count, dtype: int64
service_utilization_score: count    7032.000000
mean        2.482238
std         2.297082
min         0.000000
25%         0.000000
50%         2.000000
75%         4.200000
max         7.500000
Name: service_utilization_score, dtype: float64


In [5]:
# 5. PAYMENT RISK SCORE

# Create payment risk based on payment method and contract type
def calculate_payment_risk(row):
    risk_score = 0
    
    # Payment method risk (Electronic check is highest risk)
    if 'PaymentMethod_Electronic check' in df.columns and row['PaymentMethod_Electronic check'] == 1:
        risk_score += 3
    elif 'PaymentMethod_Mailed check' in df.columns and row['PaymentMethod_Mailed check'] == 1:
        risk_score += 2
    elif 'PaymentMethod_Bank transfer (automatic)' in df.columns and row['PaymentMethod_Bank transfer (automatic)'] == 1:
        risk_score += 1
    # Credit card (automatic) is lowest risk = 0
    
    # Contract risk (Month-to-month is highest risk)
    if 'Contract_Month-to-month' in df.columns and row['Contract_Month-to-month'] == 1:
        risk_score += 2
    elif 'Contract_One year' in df.columns and row['Contract_One year'] == 1:
        risk_score += 1
    # Two year contract is lowest risk = 0
    
    return risk_score

df['payment_risk_score'] = df.apply(calculate_payment_risk, axis=1)

print("Created payment risk score:")
print(f"payment_risk_score distribution: \n{df['payment_risk_score'].value_counts().sort_index()}")

# 6. INTERACTION FEATURES

# Contract and charges interaction
contract_cols = [col for col in df.columns if col.startswith('Contract_')]
for contract_col in contract_cols:
    contract_name = contract_col.replace('Contract_', '')
    df[f'contract_{contract_name}_charges'] = df[contract_col] * df['MonthlyCharges']

# Internet service and charges interaction
internet_service_cols = [col for col in df.columns if col.startswith('InternetService_')]
for internet_col in internet_service_cols:
    service_name = internet_col.replace('InternetService_', '')
    df[f'internet_{service_name}_charges'] = df[internet_col] * df['MonthlyCharges']

print("Created interaction features")
print(f"New interaction columns: {[col for col in df.columns if 'charges' in col and ('contract_' in col or 'internet_' in col)]}")

Created payment risk score:
payment_risk_score distribution: 
payment_risk_score
0     580
1     960
2    1309
3    1093
4    1240
5    1850
Name: count, dtype: int64
Created interaction features
New interaction columns: ['contract_Month-to-month_charges', 'contract_One year_charges', 'contract_Two year_charges', 'internet_DSL_charges', 'internet_Fiber optic_charges', 'internet_No_charges']


In [6]:
# 7. ADDITIONAL DERIVED FEATURES

# Customer lifetime value proxy
df['estimated_clv'] = df['MonthlyCharges'] * df['tenure']

# Churn risk indicators
df['high_monthly_charges'] = (df['MonthlyCharges'] > df['MonthlyCharges'].quantile(0.75)).astype(int)
df['low_tenure'] = (df['tenure'] < df['tenure'].quantile(0.25)).astype(int)
df['high_risk_combination'] = ((df['high_monthly_charges'] == 1) & (df['low_tenure'] == 1)).astype(int)

# Customer value segment
def categorize_customer_value(row):
    if row['MonthlyCharges'] >= 80 and row['tenure'] >= 24:
        return 'High_Value_Loyal'
    elif row['MonthlyCharges'] >= 80 and row['tenure'] < 24:
        return 'High_Value_New'
    elif row['MonthlyCharges'] < 80 and row['tenure'] >= 24:
        return 'Low_Value_Loyal'
    else:
        return 'Low_Value_New'

df['customer_segment'] = df.apply(categorize_customer_value, axis=1)

print("Created additional derived features:")
print(f"customer_segment distribution: \n{df['customer_segment'].value_counts()}")
print(f"high_risk_combination: {df['high_risk_combination'].value_counts()}")

Created additional derived features:
customer_segment distribution: 
customer_segment
Low_Value_New       2312
Low_Value_Loyal     2044
High_Value_Loyal    1883
High_Value_New       793
Name: count, dtype: int64
high_risk_combination: high_risk_combination
0    6880
1     152
Name: count, dtype: int64


In [7]:
# 8. FEATURE SUMMARY AND VALIDATION

# List all new features created
original_cols = pd.read_csv(data_path).columns.tolist()
new_features = [col for col in df.columns if col not in original_cols]

print(f"Original features: {len(original_cols)}")
print(f"New features: {len(new_features)}")
print(f"Total features: {len(df.columns)}")

print("\nNew features created:")
for feature in sorted(new_features):
    print(f"  - {feature}")

# Check for any missing values in new features
missing_in_new = df[new_features].isnull().sum()
if missing_in_new.sum() > 0:
    print("\nMissing values in new features:")
    print(missing_in_new[missing_in_new > 0])
else:
    print("\nNo missing values in new features ✓")

# Basic statistics for key new features
key_numeric_features = ['tenure_in_years', 'monthly_charges_to_tenure_ratio', 'total_services', 
                       'service_utilization_score', 'payment_risk_score', 'estimated_clv']

existing_key_features = [f for f in key_numeric_features if f in df.columns]
print(f"\nKey new numeric features summary:")
print(df[existing_key_features].describe())

Original features: 47
New features: 24
Total features: 71

New features created:
  - charges_vs_avg_ratio
  - contract_Month-to-month_charges
  - contract_One year_charges
  - contract_Two year_charges
  - customer_segment
  - estimated_clv
  - has_dependents
  - has_internet_service
  - has_phone_service
  - has_premium_services
  - high_monthly_charges
  - high_risk_combination
  - internet_DSL_charges
  - internet_Fiber optic_charges
  - internet_No_charges
  - is_senior_citizen
  - low_tenure
  - monthly_charges_to_tenure_ratio
  - payment_risk_score
  - service_utilization_score
  - tenure_group
  - tenure_in_years
  - total_charges_per_month
  - total_services

No missing values in new features ✓

Key new numeric features summary:
       tenure_in_years  monthly_charges_to_tenure_ratio  total_services  service_utilization_score  payment_risk_score  estimated_clv
count      7032.000000                      7032.000000     7032.000000                7032.000000         7032.000000 

In [8]:
# 9. SAVE ENGINEERED DATASET

# Ensure output directory exists
output_dir = Path('..') / 'data'
output_dir.mkdir(exist_ok=True)

# Save the feature-engineered dataset
output_path = output_dir / 'featured_telco_churn.csv'
df.to_csv(output_path, index=False)

print(f"Feature-engineered dataset saved to: {output_path}")
print(f"Final shape: {df.shape}")

# Display sample of the new dataset
print(f"\nSample of engineered features:")
sample_cols = ['tenure', 'tenure_in_years', 'tenure_group', 'MonthlyCharges', 
               'monthly_charges_to_tenure_ratio', 'total_services', 'customer_segment']
existing_sample_cols = [col for col in sample_cols if col in df.columns]
print(df[existing_sample_cols].head(10))

Feature-engineered dataset saved to: ../data/featured_telco_churn.csv
Final shape: (7032, 71)

Sample of engineered features:
   tenure  tenure_in_years tenure_group  MonthlyCharges  monthly_charges_to_tenure_ratio  total_services  customer_segment
0       1         0.083333          New           29.85                        14.925000               1     Low_Value_New
1      34         2.833333       Medium           56.95                         1.627143               2   Low_Value_Loyal
2       2         0.166667          New           53.85                        17.950000               2     Low_Value_New
3      45         3.750000       Medium           42.30                         0.919565               3   Low_Value_Loyal
4       2         0.166667          New           70.70                        23.566667               0     Low_Value_New
5       8         0.666667          New           99.65                        11.072222               3    High_Value_New
6      22    

## Feature Engineering Summary

✅ **Completed Features:**

1. **Tenure Features**: `tenure_in_years`, `tenure_group`
2. **Ratio Features**: `monthly_charges_to_tenure_ratio`, `total_charges_per_month`, `charges_vs_avg_ratio`
3. **Binary Flags**: `has_dependents`, `has_phone_service`, `has_internet_service`, `is_senior_citizen`
4. **Service Features**: `total_services`, `has_premium_services`, `service_utilization_score`
5. **Risk Features**: `payment_risk_score`, `high_risk_combination`
6. **Interaction Features**: Contract-charges and Internet-charges interactions
7. **Customer Segments**: `customer_segment` (High/Low Value × New/Loyal)
8. **Additional Features**: `estimated_clv`, `high_monthly_charges`, `low_tenure`

**Next Steps**: The feature-engineered dataset is ready for modeling in Step 4. The new features should help capture complex patterns and improve model performance for churn prediction.

**Key Insights for Modeling**:
- Ratio features will help capture the relationship between charges and loyalty
- Risk scores combine multiple indicators into actionable metrics
- Customer segmentation enables targeted business strategies
- Interaction features capture non-linear relationships between categorical and numeric variables